<a href="https://colab.research.google.com/github/DhrumilPrajapati03/VGG19_model_training_using_onera_dataset/blob/main/VGG19_onera_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import matplotlib.pyplot as plt

In [ ]:
# Define directories
ROOT_DIR = r"/content/drive/MyDrive/Onera_dataset"
IMAGE_DIR = os.path.join(ROOT_DIR, "Onera Satellite Change Detection dataset - Images","Onera Satellite Change Detection dataset - Images")
TRAIN_LABELS_DIR = os.path.join(ROOT_DIR, "Onera Satellite Change Detection dataset - Train Labels","Onera Satellite Change Detection dataset - Train Labels")
TEST_LABELS_DIR = os.path.join(ROOT_DIR, "Onera Satellite Change Detection dataset - Test Labels","Onera Satellite Change Detection dataset - Test Labels")

print(IMAGE_DIR)
print(TRAIN_LABELS_DIR)
print(TEST_LABELS_DIR)

/content/drive/MyDrive/Onera_dataset/Onera Satellite Change Detection dataset - Images/Onera Satellite Change Detection dataset - Images
/content/drive/MyDrive/Onera_dataset/Onera Satellite Change Detection dataset - Train Labels/Onera Satellite Change Detection dataset - Train Labels
/content/drive/MyDrive/Onera_dataset/Onera Satellite Change Detection dataset - Test Labels/Onera Satellite Change Detection dataset - Test Labels


In [ ]:
# Load dataset paths
def load_dataset_paths(city_folder):
  img1_dir = os.path.join(city_folder, "imgs_1_rect")
  img2_dir = os.path.join(city_folder, "imgs_2_rect")

  img1_files = sorted(os.listdir(img1_dir))
  img2_files = sorted(os.listdir(img2_dir))

  pairs = []
  for img1_file, img2_file in zip(img1_files, img2_files):
    img1_path = os.path.join(img1_dir, img1_file)
    img2_path = os.path.join(img2_dir, img2_file)
    pairs.append((img1_path, img2_path))

  return pairs

In [ ]:
# Preprocess images and labels
def preprocess_images(img1_path, img2_path, label_path, target_size=(224, 224)):
    img1 = load_img(img1_path, target_size=target_size)
    img2 = load_img(img2_path, target_size=target_size)
    label = load_img(label_path, target_size=target_size, color_mode="grayscale")

    img1 = img_to_array(img1)
    img2 = img_to_array(img2)
    label = img_to_array(label)

    img1 = preprocess_input(img1)
    img2 = preprocess_input(img2)
    label = label / 255.0  # Normalize label to [0, 1]

    return img1, img2, label

In [ ]:
# Build the VGG19-based change detection model
def build_change_detection_model(input_shape=(224, 224, 3)):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Inputs for two images
    input1 = Input(shape=input_shape)
    input2 = Input(shape=input_shape)

    # Extract features from both images
    features1 = base_model(input1)
    features2 = base_model(input2)

    # Compute absolute difference between features
    diff = tf.abs(features1 - features2)

    # Flatten and add dense layers
    x = Flatten()(diff)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Binary classification (change/no change)

    model = Model(inputs=[input1, input2], outputs=output)
    return model

# Load and preprocess the dataset
image_pairs, labels = load_dataset_paths(IMAGE_DIR, TRAIN_LABELS_DIR)
X1, X2, y = [], [], []
for img1_path, img2_path in image_pairs:
    img1, img2, label = preprocess_images(img1_path, img2_path, labels[image_pairs.index((img1_path, img2_path))])
    X1.append(img1)
    X2.append(img2)
    y.append(label)

X1 = np.array(X1)
X2 = np.array(X2)
y = np.array(y)

# Split the dataset into training and validation sets
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

# Build and compile the model
model = build_change_detection_model()
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_val, X2_val], y_val),
    batch_size=16,
    epochs=10,
    verbose=1
)

# Evaluate the model on the test dataset
test_image_pairs, test_labels = load_dataset_paths(IMAGE_DIR, TEST_LABELS_DIR)
X1_test, X2_test, y_test = [], [], []
for img1_path, img2_path in test_image_pairs:
    img1, img2, label = preprocess_images(img1_path, img2_path, test_labels[test_image_pairs.index((img1_path, img2_path))])
    X1_test.append(img1)
    X2_test.append(img2)
    y_test.append(label)

X1_test = np.array(X1_test)
X2_test = np.array(X2_test)
y_test = np.array(y_test)

test_loss, test_accuracy = model.evaluate([X1_test, X2_test], y_test, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.